In [244]:
import os
import csv
import numpy as np
import pandas as pd
import re
import nltk
import  string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import google.datalab as datalab
from sklearn.externals import joblib
from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
storage_bucket = 'gs://' + datalab.Context.default().project_id + '-datalab-workspace/'
storage_region = 'us-central1'

workspace_path = os.path.join(storage_bucket, 'twitter')

In [ ]:
!gsutil mb -c regional -l {storage_region} {storage_bucket}

In [245]:
def data_to_train():
    q1="""SELECT Distinct(text),  relevance  FROM `pecten_dataset_dev.Sentifi_Tweets` limit 4000"""
    q2="""SELECT Distinct(text),  relevance  FROM `pecten_dataset_dev.twitter_taggedtweets_collection` where relevance =-1  
limit 5000"""
    query1=client.query(q1).to_dataframe()

    query2=client.query(q2).to_dataframe()
    arg_df= pd.concat([query1, query2],axis=0)
    arg_df=arg_df.sample(frac=1).reset_index(drop=True)
    
    arg_df['relevance'] = arg_df['relevance'].convert_objects(convert_numeric=True)
    df=arg_df
    return df

In [246]:
#print(data_to_train())

In [247]:
nltk.download('stopwords')
def preprocess_tweet(text):

        stop_words = set(stopwords.words('english'))
        punct = string.punctuation

        list1=[]
        #i=0
    #for text in text1:

#        text = " ".join(re.findall(r"[a-zA-Z0-9]+", text))
        text=text.lower()
        tokens = text.split(' ')
        no_url_tokens = ['{}'.format(word) for word in tokens if 'https' not in word]
        no_url_joined = " ".join(no_url_tokens)

        filtered_tokens = [word for word in no_url_tokens if not word in stop_words]
        stemmer = PorterStemmer()
        stemmed_tokens = [stemmer.stem(word)  for word in filtered_tokens]
        dict1={'filtered_tokens': filtered_tokens, 'processed_text': stemmed_tokens}
        list1.append(dict1)
        #i+=1
        processed=pd.DataFrame(list1)

        return dict1

[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [248]:
def get_input_data(df1,train=False):
    df = df1

    X_temp = df['text'].tolist()
    #print(X_temp)
    i = 0
    X = []
    for item in X_temp:
        try:
            doc = preprocess_tweet(item)
            #print("without encode", doc)
            doc=[words.encode("latin-1","ignore") for words in doc['filtered_tokens']]
            #print("encoded", doc)
            X.append(doc)
            i += 1
            
        except Exception as e:
            print(e)
    #print("formatted",'\n',X)

    if "relevance" in df.columns:
        y = np.array(df["relevance"])
    else:
        print(Y_NONE)
    #print(X,y)

    vectorizer = TfidfVectorizer(sublinear_tf=True,
                                    # max_df=1,
                                     stop_words='english',
                                     ngram_range=(1, 2), lowercase=False)
#                                     tokenizer=tokenizer.tokenize)
    if train:
        X_train = vectorizer.fit_transform(X_temp)
        joblib.dump(vectorizer, str(os.path.join(r"","tf_vectorizer.plk")))
        return X, X_train, y
    else:
        vectorizer = joblib.load(str(os.path.join(r"","tf_vectorizer.plk")))
        X_test = vectorizer.transform(X)
        return X, X_test,y

In [249]:

def train_model(X_train, y_train,model_name):
    if model_name == "random_forest.plk":
        clf = RandomForestClassifier(n_estimators=500, class_weight="balanced")
        clf.fit(X_train, y_train)
        joblib.dump(clf, str(os.path.join(r"", model_name)))
    if model_name == "SV_classifier.plk":
        clf = SVC(kernel="rbf", gamma=5, C=5.0)
        clf.fit(X_train, y_train)
        joblib.dump(clf, str(os.path.join(r"", model_name)))
    return clf


In [250]:
def main_func(model_name,train=False):
    df1=data_to_train()
    X_original, X, y = get_input_data(df1,train)

    if train:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        print("Training model")
        classifier = train_model(X_train,y_train, model_name)
        predicted = classifier.predict(X_test)
        print("Accuracy: {}".format(classifier.score(X_test,y_test)))
    else:
        classifier = joblib.load(str(os.path.join(r"", model_name)))
        predicted = classifier.predict(X)
    return X_original,X,y,predicted

In [251]:
def model():
    train=True
    X, x_vector,y,pred = main_func(
          "random_forest.plk",train)
#    return X, x_vector,y,pred

In [252]:
model()

/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


Training model
Accuracy: 0.9661111111111111


In [ ]:
!gcloud components update



Your current Cloud SDK version is: 212.0.0
You will be upgraded to version: 220.0.0

┌────────────────────────────────────────────────────────────────────────────┐
│                     These components will be updated.                      │
├─────────────────────────────────────────────────────┬────────────┬─────────┤
│                         Name                        │  Version   │   Size  │
├─────────────────────────────────────────────────────┼────────────┼─────────┤
│ BigQuery Command Line Tool                          │     2.0.34 │ < 1 MiB │
│ BigQuery Command Line Tool (Platform Specific)      │     2.0.34 │ < 1 MiB │
│ Cloud SDK Core Libraries                            │ 2018.10.08 │ 8.6 MiB │
│ Cloud SDK Core Libraries (Platform Specific)        │ 2018.09.24 │ < 1 MiB │
│ Cloud Storage Command Line Tool                     │       4.34 │ 3.5 MiB │
│ Cloud Storage Command Line Tool (Platform Specific) │       4.34 │ < 1 MiB │
│ gcloud Alpha Commands                     

        ◆ --show-occurrences-from For more information about occurrences, see
          <https://cloud.google.com/container-registry/docs/container-analysis>.

  Firebase Test Lab
      ▪ Fixed bug where --environment-variables did not work in the 217.0.0
        release.

  Kubernetes Engine
      ▪ Added --metadata and --metadata-from-file flags to gcloud
        <node-pools|clusters> create.
      ▪ Added --internal-ip flag to gcloud beta container clusters
        get-credentials.

    Subscribe to these release notes at
    https://groups.google.com/forum/#!forum/google-cloud-sdk-announce
    (https://groups.google.com/forum/#!forum/google-cloud-sdk-announce).

217.0.0 (2018-09-18)
  Breaking Changes
      ▪ **(Cloud Services)** Renamed --reserved-ranges to ranges in gcloud
        beta services vpc-peerings.

  App Engine
      ▪ Updated the Java SDK to version 1.9.65. Please visit the following
        release notes for details:
        https://cloud.google.com/appengine/docs/ja

        clusters create to beta.
      ▪ Promoted --max-pods-per-node flag of gcloud container node-pools
        create to beta.
      ▪ Modified --monitoring-service flag of gcloud containers clusters
        update to enable Google Cloud Monitoring service with Kubernetes-native
        resource model.
      ▪ Modified --logging-service flag of gcloud containers clusters update
        to enable Google Cloud Logging service with Kubernetes-native resource
        model.
      ▪ Modified output of gcloud beta container clusters list for DEGRADED
        clusters to include reason for degradation.
      ▪ Added --enable-private-nodes and --enable-private-endpoint to gcloud
        beta container clusters create.
      ▪ Deprecated --private-cluster flag of gcloud beta container clusters
        create; use --enable-private-nodes instead.

    Subscribe to these release notes at
    https://groups.google.com/forum/#!forum/google-cloud-sdk-announce
    (https://groups.google.com/forum/#